In [2]:
%pip install -q -U -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


### Utilities

In [3]:
import matplotlib.pyplot as plt
import networkx as nx

def quick_plot(G):

    pos = nx.spring_layout(G)
    nx.draw(G, pos)
    nx.draw_networkx_labels(d2g.graph, pos)

    plt.show()


In [4]:
from pyvis.network import Network
def nice_plot(graph, file_name):
        net = Network(
            notebook=True,
            cdn_resources="in_line",
            bgcolor="#222222",
            font_color="white",
            height="750px",
            width="100%",
        )
        # Convert the NetworkX graph to a PyVis graph
        for n in graph.nodes(data=True):
            print(n)
            n[1]['label']=n[0]+' '+n[1]['type'] 

        net.from_nx(graph)

        # Render the graph in the Jupyter notebook
        net.show_buttons()
        net.show(file_name + ".html")


In [5]:
def find_nodes_by_attribute(G, attribute_name, attribute_value=None):
    found_nodes = []
    for node, attrs in G.nodes(data=True):
        if attribute_name in attrs:
            if attribute_value is None or attrs[attribute_name] == attribute_value:
                found_nodes.append(node)
    return found_nodes


In [6]:
def nearest_neighbors(G, start_node, depth=2):

    bfs_edges = nx.bfs_edges(G, start_node, depth_limit=depth)
    visited_nodes = set([start_node])
    edges_list = []

    for u, v in bfs_edges:
        visited_nodes.add(v)
        edges_list.append((u, v))

    print("Node Attributes:")
    for node in visited_nodes:
        pp.pprint(f"Node {node}")
        for attr, value in G.nodes[node].items():
            print(f"\t{attr}: {value}")

    print("\nEdge Attributes:")
    for u, v in edges_list:
        if G.has_edge(u, v):
            pp.pprint(f"Edge ({u}, {v}): {G[u][v]}")


In [7]:
def print_element(element):
    print("Type of element:", type(element))
    print("ID:", element.id)
    print(element.text)
    print("Metadata:")
    pp.pprint(element.metadata.to_dict())


In [8]:
import networkx as nx
def print_node_attributes(child_node):
    for parent, child in nx.bfs_predecessors(G.reverse(), child_node):
        # Print node attributes
        child_attributes = G.nodes[child]
        parent_attributes = G.nodes[parent]

        print(f"Child Node {child} Attributes: {child_attributes}")
        print(f"Parent Node {parent} Attributes: {parent_attributes}")

        # Print edge attributes
        edge_attributes = G[parent][child]
        print(f"Edge from {parent} to {child} Attributes: {edge_attributes}")
        print("-" * 50)

    # Additionally, print the attributes of the starting node (child_node)
    print(f"Starting Node {child_node} Attributes: {G.nodes[child_node]}")


In [9]:
def find_element(id:str):
    for element in document["elements"]:
        if element.id == id:
            return element
    return None


In [10]:
def get_fanout_edges_and_nodes(graph, source_node):
    fanout_edges = list(nx.bfs_edges(graph, source=source_node))
    fanout_nodes = set([node for edge in fanout_edges for node in edge])
    print("Fanout edges:", fanout_edges)
    print("Fanout nodes:", fanout_nodes)
    return fanout_edges, fanout_nodes


### Converts Doc to Graph

In [11]:
from unstructured.partition.pdf import partition_pdf
from doc2graph import Doc2Graph
from tqdm.notebook import tqdm
import os
from glob import glob

def document_processor(directory, pattern):
    for x in os.walk(directory):
        pdf_files = glob(os.path.join(x[0], pattern))
        for file_name in tqdm(pdf_files):
            elements = partition_pdf(
                filename=file_name,
                extract_images_in_pdf=True,
                infer_table_structure=True,
                max_characters=2000,
                new_after_n_chars=1700,
                extract_image_block_output_dir="images/",
            )
            yield {"file": file_name, "elements": elements}
            
for document in document_processor("./data", "*.pdf"):
    d2g = Doc2Graph(document["file"], infer_hierarchy=False)
    for element in document["elements"]:
        d2g.process_element(element)
    break

d2g.plot_graph("doc2graph")
G = d2g.graph



  0%|          | 0/1 [00:00<?, ?it/s]

2024-10-23 10:27:29,897 - pikepdf._core - INFO - pikepdf C++ to Python logger bridge initialized
2024-10-23 10:28:38,437 - unstructured_inference - INFO - Reading PDF for file: ./data/MSNA20180726707877.pdf ...
2024-10-23 10:30:03,924 - unstructured_inference - INFO - Loading the Table agent ...
2024-10-23 10:30:03,925 - unstructured_inference - INFO - Loading the table structure model ...
2024-10-23 10:30:04,128 - timm.models._builder - INFO - Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)
2024-10-23 10:30:04,205 - timm.models._hub - INFO - [timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2024-10-23 10:30:04,240 - timm.models._builder - INFO - Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when in

doc2graph.html


In [12]:
from nx2document import NxToDocuments
from dotenv import load_dotenv
load_dotenv()
graphrag = NxToDocuments(connect_to_database=True, reset_store=True)
for doc in graphrag.nx_to_documents(G):
    print(doc)


2024-10-23 11:04:12,774 - cassandra.policies - INFO - Using datacenter 'us-east-2' for DCAwareRoundRobinPolicy (via host 'e9b06722-b7b7-4d7d-9ba0-801344f200de-us-east-2.db.astra.datastax.com:29042:0669df0f-031f-383e-b452-67a9679bbc6a'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
/Users/pedropacheco/Projects/demos/unstructured-raggraph/nx2document.py:24: LangChainBetaWarning: The class `CassandraGraphVectorStore` is in beta. It is actively being worked on, so the API may change.
  self.content_centric_store = CassandraGraphVectorStore(
2024-10-23 11:04:14,537 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-23 11:04:15,897 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-23 11:04:16,233 - cassandra.protocol - WARNING - Server warning: Detected collection link_to_tags with 85 items, greater than the maximum recommended (20)
20

page_content='n/a' metadata={'type': 'document', 'links': [Link(kind='document', direction='out', tag='f41b015e6c9ed4b84dd25e6e631d8721'), Link(kind='document', direction='out', tag='f070e7f9ebc24af8414c6d08f4726731'), Link(kind='document', direction='out', tag='e563f0d09a5568da64351a5bbb67bebe'), Link(kind='document', direction='out', tag='407006d14a6acc3a5011a15d6e2be801'), Link(kind='document', direction='out', tag='fd96d7cf8cadabac3bb0f0982813e636'), Link(kind='document', direction='out', tag='6b15dfb496b0a05db39b26b10a978c21'), Link(kind='document', direction='out', tag='4f64898538fdbc30d889389fde660884'), Link(kind='document', direction='out', tag='5aa1120c9bed65e09c4331e818525fad'), Link(kind='document', direction='out', tag='ef563c511cb861c7073874492adcc86b'), Link(kind='document', direction='out', tag='7de6f52965131771c63856a6bbe4c14a'), Link(kind='document', direction='out', tag='95df813abe6e7dc7a867c8be00ba1fd2'), Link(kind='document', direction='out', tag='ebd067809a889179b

2024-10-23 11:04:16,618 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


page_content='Morgan Stanley | esearcs' metadata={'type': 'section', 'links': [Link(kind='section', direction='out', tag='7c0b21986ecd138b750a748b98649e0d'), Link(kind='section', direction='out', tag='304702ef6f9468402bc3bd218f8e9bec')]}
page_content='August 1, 2018 02:18 AM GMT' metadata={'type': 'narravite_text', 'links': []}


2024-10-23 11:04:17,012 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


...

page_content='COMPANY (TICKER) RATING (AS OF) PRICE* (07/31/2018) Katy L. Huberty, CFA Apple, Inc. (AAPL.O) O (05/26/2009) $190.29 CDW Corporation (CDW.O) E (08/06/2013) $84.09 Electronics for Imaging Inc (EFII.O) U (08/03/2017) $34.12 Hewlett Packard Enterprise (HPE.N) HP Inc. (HPQ.N) O (01/23/2018) E (01/23/2018) $15.44 $23.08 IBM (IBM.N) O (02/18/2016) $144.93 NCR Corp. (NCR.N) E (09/25/2011) $27.92 NetApp Inc (NTAP.O) E (01/23/2018) $77.52 Nutanix Inc (NTNX.O) Pure Storage Inc (PSTG.N) E (05/10/2017) E (05/18/2017) $48.89 $21.66 Seagate Technology (STX.O) O (04/09/2018) $52.62 Teradata (TDC.N) E (01/23/2018) $38.29 Xerox Corp (XRX.N) ++ $25.97 Yuuji Anderson Fitbit Inc (FIT.N) Garmin Ltd (GRMN.O) U (04/01/2018) E (01/07/2015) $5.93 $62.45 GoPro Inc (GPRO.O) U (01/23/2018) $5.84' metadata={'type': 'table', 'links': []}
page_content='Stock Ratings are subject to change. Please see latest research for each company.' metadata={'type': 'narravite_text', 'links': []}


2024-10-23 11:05:37,777 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-23 11:05:37,956 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


page_content='* Historical prices are not split adjusted.' metadata={'type': 'narravite_text', 'links': []}
page_content='© 2018 Morgan Stanley' metadata={'type': 'section', 'links': [Link(kind='section', direction='out', tag='b1993a85e531ebcc7c51396004dce994')]}


2024-10-23 11:05:38,471 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


page_content='UPDATE' metadata={'type': 'image', 'links': []}


In [15]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

retriever = graphrag.content_centric_store.as_retriever()
docs = retriever.invoke("what is the price target range for the stock of Apple Inc. (AAPL)?")

for doc in docs:
    print(doc.page_content)
    print("\n")


2024-10-23 11:10:11,452 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


$  400  350  300  $335.00 (+76%)  250  200  $190.29  $232.00 (+22%)  150  $145.00 (-24%)  100  50  0  Jul-16  Jan-17  Price Target (Jul-19)  Jul-17  Jan-18  Historical Stock Performance  Jul-18  Jan-19  Current Stock Price  Jul-19  WARNINGDONOTEDIT_RRS4RL~AAPL.O~ 


Price Target


Exhibit 3: Our sum-of-the-parts (SOTP) analysis drives our Apple price target of $232 (unchanged), which represents 15.7x CY19 EPS.


Apple, Inc. CAAPL.O) - As of 7/31/18 in USD Industry : IT Hardware $040 +0 ahd 3 aie 2 toe 200 ate 20 ememe! 200 177 ygp eee =u! 16: 154 moe 155 is2 148 150 1 a Meee ely . —L 123 mr <a 120 be re 100 0 08704 13704 08704 080 2015 2016 2017 2016 Stock Rating History: 6/1/15 : O/C? 1/25/16 2 OM Price Target History: 7/22/15 2: 1557 10711715 : 1627 1026715 2 1527 12/13/15 2 1437 1726716 = 135; 4/27/16 : 1207 7726716 2 1237 10/14/16 & 1247 1025716 5 1487 1451717 2 1507 221717 = 1547 4719/17 2 1617 S/i7/i7 : 17? e F716717 2 1627 B719/17 & 1947 10710717 5 1997 11/2717 2 2007 1/23/18 : 

In [14]:
# Define the prompt template
prompt_template = """
You are an expert financial analyst. Based on the following documents, provide a detailed analysis of Apple Inc. (AAPL) and its stock price target range.:

{documents}

Your analysis:
"""

# Create a prompt
prompt = PromptTemplate.from_template(prompt_template)

# Initialize the OpenAI LLM
chat = ChatOpenAI(model="gpt-4")
messages = [HumanMessage(content=prompt.format(documents=docs))]
response = chat(messages)
print(response.content)


/var/folders/hr/bsttdbns4nn9l1g54smlh_mr0000gn/T/ipykernel_70455/1350731166.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model="gpt-4")
/var/folders/hr/bsttdbns4nn9l1g54smlh_mr0000gn/T/ipykernel_70455/1350731166.py:16: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat(messages)
2024-10-23 11:09:10,713 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Based on the provided documents, Apple Inc.'s stock has seen significant changes in its price target over the years. The price target has fluctuated from a low of $145.00, indicating a -24% change, to a high of $335.00, suggesting a +76% change. As of the latest data available in the documents, the price target for AAPL is $232.00, which depicts a +22% change.

The sum-of-the-parts (SOTP) analysis has been used to derive the price target of $232.00, which is equivalent to 15.7 times the Calendar Year 2019 (CY19) Earnings Per Share (EPS). This method involves evaluating each segment of the company to calculate the total business value. 

Historically, the stock price has seen various ratings, ranging from Overweight (O) to Equal-weight (E) and Underweight (U). However, no current specific rating is provided in the documents. 

The documents suggest that Apple Inc. operates in the IT Hardware industry, but there is no specific industry view or rating given. 

Given the historical stock p